In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False


In [5]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

# Sequence 1

In [6]:
a = 0.25
num_params=9

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/3) )
c1,c2 = calc_c(nobs)

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.50, 5th: 3.92, 10th: 4.78, 30th: 7.21, 50th: 9.29, 70th: 11.77, 90th: 16.04, 95th: 18.14, 99th: 22.71
bonus term 0.00935751153768674 // variance stats 0.33762379479580645 //part1 9.999999999999998 //part2 0.0009357511537686742
-2.229542447842249 -1.396442590412838 2.1113859938797064
-2.248257470917623 -1.396442590412838 2.0926709708043325
---
Quantiles: 1th: 2.73, 5th: 4.21, 10th: 5.08, 30th: 7.42, 50th: 9.51, 70th: 12.32, 90th: 17.00, 95th: 19.55, 99th: 25.52
bonus term 0.008939168520454439 // variance stats 0.3534390071792148 //part1 9.999999999999998 //part2 0.0008939168520454441
-1.8505926621113777 -1.4864180239541265 1.7767281245490307
-1.8684709991522868 -1.4864180239541265 1.7588497875081217
---
Quantiles: 1th: 2.56, 5th: 4.00, 10th: 4.99, 30th: 7.43, 50th: 9.65, 70th: 12.13, 90th: 16.81, 95th: 19.28, 99th: 25.00
bonus term 0.011118446160410548 // variance stats 0.2841010023527295 //part1 9.999999999999998 //part2 0.001111844616041055
-1.5931344554646125 0.7899

In [7]:
nobs = 250
c1,c2 = calc_c(nobs)

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.46, 5th: 3.70, 10th: 4.60, 30th: 6.89, 50th: 9.03, 70th: 11.62, 90th: 16.52, 95th: 19.28, 99th: 26.37
bonus term 0.4188026352291261 // variance stats 0.14673430407697274 //part1 6.299605249474365 //part2 0.06648077437297689
-1.299417735412158 1.7082169647640082 2.2355684037882484
-2.1370230058704105 1.7082169647640082 1.3979631333299962
---
Quantiles: 1th: 2.43, 5th: 3.77, 10th: 4.81, 30th: 7.49, 50th: 9.77, 70th: 12.54, 90th: 17.63, 95th: 20.85, 99th: 27.14
bonus term 0.3622870709030431 // variance stats 0.18271720281214626 //part1 6.299605249474365 //part2 0.057509487746596824
-1.3812607510541628 1.441564514040623 2.3285961468391596
-2.1058348928602486 1.441564514040623 1.6040220050330736
---
Quantiles: 1th: 2.89, 5th: 4.48, 10th: 5.52, 30th: 8.31, 50th: 10.75, 70th: 13.84, 90th: 18.98, 95th: 22.34, 99th: 29.31
bonus term 0.18352570343776392 // variance stats 0.4000458634535611 //part1 6.299605249474365 //part2 0.029132889470031027
-1.8604118038699742 3.224659187426

# Sequence 2

In [8]:
a = 0.25
num_params=9

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/7) )
c1,c2 = calc_c(nobs)

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.43, 5th: 3.90, 10th: 4.83, 30th: 7.27, 50th: 9.62, 70th: 12.45, 90th: 17.42, 95th: 20.37, 99th: 27.80
bonus term 0.00387149090028811 // variance stats 0.3588805545054238 //part1 2.2007102102809872 //part2 0.001759200680853749
-1.9066135481819573 0.8975808701213563 1.9122080588250174
-1.9143565299825336 0.8975808701213563 1.9044650770244411
---
Quantiles: 1th: 2.63, 5th: 3.98, 10th: 4.81, 30th: 7.13, 50th: 9.34, 70th: 11.99, 90th: 16.73, 95th: 19.47, 99th: 26.44
bonus term 0.004357997097911828 // variance stats 0.3187462213289771 //part1 2.2007102102809872 //part2 0.0019802684958486185
-2.00675090004795 0.40286804178771335 1.833154680764075
-2.0154668942437737 0.40286804178771335 1.8244386865682516
---
Quantiles: 1th: 2.58, 5th: 3.76, 10th: 4.60, 30th: 6.91, 50th: 8.98, 70th: 11.38, 90th: 16.05, 95th: 18.75, 99th: 23.98
bonus term 0.003986229692738999 // variance stats 0.34853240805585517 //part1 2.2007102102809872 //part2 0.0018113378463537168
-1.8121442673779318 0.93

In [9]:
nobs = 250
c1,c2 = calc_c(nobs)

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.39, 5th: 3.73, 10th: 4.67, 30th: 7.05, 50th: 9.27, 70th: 11.91, 90th: 16.44, 95th: 19.48, 99th: 25.21
bonus term 0.07145799035105904 // variance stats 0.30972628692221676 //part1 2.2007102102809872 //part2 0.03247042251052912
-1.8684448848312265 2.1538289748641906 2.0517663638069243
-2.0113608655333444 2.1538289748641906 1.9088503831048063
---
Quantiles: 1th: 2.67, 5th: 3.88, 10th: 4.80, 30th: 7.40, 50th: 9.77, 70th: 12.46, 90th: 17.60, 95th: 20.47, 99th: 26.67
bonus term 0.06286804213247667 // variance stats 0.3785929652517619 //part1 2.2007102102809872 //part2 0.028567160655127627
-1.9879599985678171 0.4318564190047818 1.8596652625122032
-2.1136960828327704 0.4318564190047818 1.7339291782472497
---
Quantiles: 1th: 2.78, 5th: 4.14, 10th: 5.22, 30th: 7.87, 50th: 10.41, 70th: 13.66, 90th: 19.66, 95th: 23.38, 99th: 31.11
bonus term 0.06789162730357796 // variance stats 0.3906771444648997 //part1 2.2007102102809872 //part2 0.03084987154892581
-1.7129901724662384 2.232374

# a  = .25, k= 9

In [10]:
a = 0.25
num_params=9
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )


In [11]:
nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.60, 5th: 3.98, 10th: 4.87, 30th: 7.33, 50th: 9.48, 70th: 12.16, 90th: 17.07, 95th: 19.92, 99th: 26.09
bonus term 0.07705254019167865 // variance stats 0.4120235690151082 //part1 3.017088168272582 //part2 0.025538710138456006
-1.727640368075886 0.4923263221729579 2.0366810137039764
-1.8817454484592433 0.4923263221729579 1.8825759333206191
---
Quantiles: 1th: 2.61, 5th: 3.84, 10th: 4.60, 30th: 6.98, 50th: 9.02, 70th: 11.63, 90th: 16.55, 95th: 19.41, 99th: 25.98
bonus term 0.08262036248582161 // variance stats 0.37166084865269927 //part1 3.017088168272582 //part2 0.027384139235521732
-1.6821611151140203 0.7398398873207715 2.0046003506793935
-1.8474018400856638 0.7398398873207715 1.8393596257077502
---
Quantiles: 1th: 2.58, 5th: 3.82, 10th: 4.76, 30th: 7.25, 50th: 9.51, 70th: 12.26, 90th: 17.26, 95th: 20.35, 99th: 26.02
bonus term 0.09520109599059742 // variance stats 0.3349496181847244 //part1 3.017088168272582 //part2 0.03155396550612053
-1.7838359535315313 1.0009763974

In [12]:

nobs = 500
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.77, 5th: 4.05, 10th: 5.01, 30th: 7.42, 50th: 9.53, 70th: 11.93, 90th: 16.26, 95th: 18.77, 99th: 23.94
bonus term 0.0033231286629299703 // variance stats 0.46595633100327843 //part1 3.465724215775732 //part2 0.0009588554818653265
-1.931336320430302 1.313514426334218 2.1069306901086877
-1.9379825777561621 1.313514426334218 2.1002844327828276
---
Quantiles: 1th: 2.97, 5th: 4.23, 10th: 5.17, 30th: 7.72, 50th: 9.96, 70th: 12.81, 90th: 17.99, 95th: 20.81, 99th: 26.34
bonus term 0.004729335345767005 // variance stats 0.3272772707626654 //part1 3.465724215775732 //part2 0.0013646023316683433
-1.9809228009942814 0.5824960136390647 1.9921076546115046
-1.9903814716858153 0.5824960136390647 1.9826489839199708
---
Quantiles: 1th: 3.09, 5th: 4.59, 10th: 5.66, 30th: 8.43, 50th: 10.99, 70th: 14.07, 90th: 19.29, 95th: 22.34, 99th: 28.87
bonus term 0.00423265634759941 // variance stats 0.3657339408058934 //part1 3.465724215775732 //part2 0.0012212905828838478
-1.877118257812293 1.29626

In [13]:
nobs = 100
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.14, 5th: 3.39, 10th: 4.21, 30th: 6.53, 50th: 8.54, 70th: 11.14, 90th: 16.19, 95th: 19.17, 99th: 26.11
bonus term 0.14323176965679757 // variance stats 0.2677095828617784 //part1 2.51188643150958 //part2 0.057021594551437946
-1.5504612344753244 -0.12157223305070627 2.2168647447262484
-1.8369247737889194 -0.12157223305070627 1.9304012054126531
---
Quantiles: 1th: 2.91, 5th: 4.35, 10th: 5.31, 30th: 8.16, 50th: 10.69, 70th: 14.07, 90th: 20.60, 95th: 25.30, 99th: 35.34
bonus term 0.09601445414702177 // variance stats 0.5184030399727766 //part1 2.51188643150958 //part2 0.03822404267270933
-1.7667137900091365 -1.142732407298438 1.959984624687571
-1.9587426983031802 -1.142732407298438 1.7679557163935273
---
Quantiles: 1th: 2.44, 5th: 3.85, 10th: 4.83, 30th: 7.33, 50th: 9.73, 70th: 12.87, 90th: 18.72, 95th: 22.16, 99th: 31.41
bonus term 0.12479647054800032 // variance stats 0.3581356074147606 //part1 2.51188643150958 //part2 0.04968236978492726
-1.8159638638047342 -0.401278623

# a  = .25, k= 4, something not right?

In [14]:
a = 0.25
num_params=4



In [15]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.96 & 0.96 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.02 & 0.01   \\
Model 2 & 0.04 & 0.04 & 0.04 & 0.02 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.962, 0.002, 0.036]), array([0.962, 0.002, 0.036]), array([0.962, 0.002, 0.036]), array([0.964, 0.02 , 0.016]), array([0.972, 0.016, 0.012]), array([0.984, 0.008, 0.008]), -1.7992081929910861, 5.416545551668353, 5.715396300306833)


# a  = .25, K = 19

In [16]:
a = 0.25
num_params=19


In [17]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.70 & 0.80 & 0.72 & 0.97 & 0.98 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.30 & 0.20 & 0.28 & 0.02 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.7, 0. , 0.3]), array([0.804, 0.   , 0.196]), array([0.722, 0.   , 0.278]), array([0.97 , 0.008, 0.022]), array([0.976, 0.006, 0.018]), array([0.972, 0.008, 0.02 ]), -9.658578371553618, 6.304913603857928, 6.645671173756523)


# evidence of power

In [18]:
num_params= 9
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.87 & 0.54 & 0.59 & 0.56   \\
Model 1 & 0.17 & 0.16 & 0.13 & 0.46 & 0.41 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.832, 0.166, 0.002]), array([0.834, 0.164, 0.002]), array([0.866, 0.132, 0.002]), array([0.536, 0.464, 0.   ]), array([0.592, 0.408, 0.   ]), array([0.562, 0.438, 0.   ]), 6.338213059649856, 4.8970852060510754, 5.313946762038993)


In [19]:
nobs = 500
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.83 & 0.83 & 0.50 & 0.56 & 0.54   \\
Model 1 & 0.18 & 0.17 & 0.17 & 0.50 & 0.44 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.822, 0.178, 0.   ]), array([0.826, 0.174, 0.   ]), array([0.83, 0.17, 0.  ]), array([0.502, 0.498, 0.   ]), array([0.556, 0.444, 0.   ]), array([0.54, 0.46, 0.  ]), 6.953548686077008, 5.2620129910545534, 5.465950037597665)


In [20]:

nobs = 100
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.78 & 0.84 & 0.81 & 0.54 & 0.58 & 0.55   \\
Model 1 & 0.22 & 0.16 & 0.19 & 0.46 & 0.42 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.776, 0.224, 0.   ]), array([0.84, 0.16, 0.  ]), array([0.812, 0.188, 0.   ]), array([0.544, 0.456, 0.   ]), array([0.58, 0.42, 0.  ]), array([0.55, 0.45, 0.  ]), 6.433835593599485, 5.325069811279112, 5.038012843932768)


In [21]:
num_params= 19
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.99 & 0.98 & 0.64 & 0.73 & 0.65   \\
Model 1 & 0.02 & 0.01 & 0.01 & 0.36 & 0.27 & 0.35   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.976, 0.016, 0.008]), array([0.988, 0.008, 0.004]), array([0.98 , 0.014, 0.006]), array([0.64, 0.36, 0.  ]), array([0.732, 0.268, 0.   ]), array([0.648, 0.352, 0.   ]), 1.2179547417213819, 5.366534685350651, 6.098171859769255)


In [22]:
num_params= 4
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.59 & 0.59 & 0.62 & 0.45 & 0.48 & 0.54   \\
Model 1 & 0.41 & 0.41 & 0.38 & 0.55 & 0.52 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.586, 0.414, 0.   ]), array([0.586, 0.414, 0.   ]), array([0.618, 0.382, 0.   ]), array([0.448, 0.552, 0.   ]), array([0.478, 0.522, 0.   ]), array([0.538, 0.462, 0.   ]), 9.235599195914721, 5.047613387342013, 4.875736214626621)


# evidence of power 2

In [23]:
num_params= 9
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.08 & 0.19 & 0.14 & 0.51 & 0.57 & 0.54   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.92 & 0.81 & 0.86 & 0.49 & 0.43 & 0.46   \\
\hline
\end{tabular}
(array([0.084, 0.   , 0.916]), array([0.19, 0.  , 0.81]), array([0.136, 0.   , 0.864]), array([0.512, 0.   , 0.488]), array([0.568, 0.   , 0.432]), array([0.542, 0.   , 0.458]), -15.101549976216862, 5.08919251678837, 5.394325586243468)


In [24]:
nobs = 500
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.10 & 0.15 & 0.12 & 0.55 & 0.62 & 0.57   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.90 & 0.85 & 0.88 & 0.45 & 0.38 & 0.43   \\
\hline
\end{tabular}
(array([0.096, 0.   , 0.904]), array([0.154, 0.   , 0.846]), array([0.116, 0.   , 0.884]), array([0.55, 0.  , 0.45]), array([0.624, 0.   , 0.376]), array([0.57, 0.  , 0.43]), -14.747327901065185, 4.838927123530901, 5.425322108271465)


In [25]:
nobs = 100
c1,c2 = calc_c(nobs)



a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.07 & 0.44 & 0.16 & 0.54 & 0.58 & 0.54   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.93 & 0.56 & 0.84 & 0.46 & 0.42 & 0.46   \\
\hline
\end{tabular}
(array([0.072, 0.   , 0.928]), array([0.436, 0.   , 0.564]), array([0.164, 0.   , 0.836]), array([0.538, 0.   , 0.462]), array([0.58, 0.  , 0.42]), array([0.544, 0.   , 0.456]), -14.830681663719801, 4.818957871943981, 5.049769104551095)


In [26]:
num_params= 19
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.56 & 0.01 & 0.65 & 0.72 & 0.66   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.44 & 0.99 & 0.35 & 0.28 & 0.34   \\
\hline
\end{tabular}
(array([0.004, 0.   , 0.996]), array([0.562, 0.   , 0.438]), array([0.008, 0.   , 0.992]), array([0.65, 0.  , 0.35]), array([0.722, 0.   , 0.278]), array([0.656, 0.   , 0.344]), -20.3093817605501, 5.852702893435854, 6.162771706497528)


In [27]:
num_params= 4
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.21 & 0.22 & 0.25 & 0.45 & 0.49 & 0.54   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.79 & 0.78 & 0.75 & 0.55 & 0.51 & 0.46   \\
\hline
\end{tabular}
(array([0.214, 0.   , 0.786]), array([0.218, 0.   , 0.782]), array([0.252, 0.   , 0.748]), array([0.452, 0.   , 0.548]), array([0.492, 0.   , 0.508]), array([0.536, 0.   , 0.464]), -12.192186908171703, 4.653657688259495, 4.875359110182073)
